In [8]:
## This script is to calculate percentages of Volume per Service Task Type (STT) for each Service Type (ST), ie. we are aggregating at Service Type level, this is to allow business stakeholders to work at a higher level of granularity (ST instead of STT) to save them time.
# Consumes two files but also produces the file for #2: 
# 1. The file with the individual volume per Service Task Type per Service Type
# 2. The updated file with aggregated volume at Service Type level (not at Service Task Type level ) coming from stakeholders, which this notebook also produces.   
# Once stakholder returns the updated file, we will use it to calculate the updated individual volume based on the individual percentages per each Service Task Type according to this logic:
#   2.1) If we had volume for a given service task type (STT), then calculate the percentage of this STT for the ST they belong to and refer to the new volume for the group, else, 
#   2.2) If we have zeros in volume at STT level, then use the general updated volume at the ST level and distribute evenly (prorate) for each STT in that ST group.

## import libraries
import pandas as pd
import numpy as np



In [ ]:
 
# Load csv file with all the data that has all the combinations for the Fiscal Year 
# make first row as header
df = pd.read_csv('DetailedForecast.csv', header=0)

## print headers
print(df.columns)

# Convert Volume column to numeric
df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')


In [15]:
# Copy dataframe, to do aggregation up to the Service Type level, this will be for the stakeholder to update the volume at this level
dfagg = df.copy()
# Calculate the Volume for each Fiscal Year, Month, Country, ServiceLine, and ServiceType

dfagg = df.groupby(['Fiscal Year', 'Month', 'Country', 'ServiceLine', 'ServiceType']).agg({'Volume': 'sum'}).reset_index()

# save the aggregated data to a csv file
dfagg.to_csv('Forecast_Aggregated.csv', index=False)

#Send this file to stakholder to have them work on the Volume at the Service Type level





In [ ]:
## Group by Fiscal Year, Month, Country, Service Line, and ServiceType and calculate the Percentage that each ServiceTaskType contributes to each ServiceType group based on the Volume
df['Percentage'] = df['Volume'] / df.groupby(['Fiscal Year', 'Month', 'Country', 'ServiceLine', 'ServiceType'])['Volume'].transform('sum') * 100

## print the result
print(df)




In [6]:
## Intermediate step for debugging, save back as csv
df.to_csv('Forecast with Percentages.csv', index=False)


In [ ]:
# Once stakeholder reviews and updates the aggregated Volume at ServiceType level, we will use this file to calculate the updated Volume at ServiceTaskType level based on the percentages calculated above 

# Open the updated file
dfagg = pd.read_csv('Forecast_Aggregated.csv', header=0)

# print headers
print(dfagg.columns)



In [ ]:
# view first rows
dfagg.head()


In [ ]:
# This is the most important part of the code! 
#Check for zero, null, or NaN values in the 'Percentage' column of 'df'
if df['Percentage'].isnull().any() or (df['Percentage'] == 0).any() or (df['Percentage'] == np.nan).any():
    df['Percentage'] = df['Percentage'].replace(np.nan, 0)  # Replace NaN values with 0
    df['Percentage'] = df['Percentage'].replace(np.inf, 0)  # Replace inf values with 0
    df['Percentage'] = df['Percentage'].fillna(0)  # Replace null values with 0
    print("Warning: The 'Percentage' column in the 'df' DataFrame contains zero, null, or NaN values. These values will be treated as zero in the proration calculation.")

# Merge df and dfagg on the specified columns
merged_df = pd.merge(df, dfagg, on=['Fiscal Year', 'Month', 'Country', 'Program', 'ServiceLine', 'ServiceType'], suffixes=('_df', '_dfagg'))

#  If we have zeros in volume at STT level, then use the general updated volume at the ST level and distribute evenly (prorate) for each STT in that ST group.
grouped = merged_df.groupby(['Fiscal Year', 'Month', 'Country', 'ServiceLine', 'ServiceType'])['Volume_df']
merged_df['Prorated Volume'] = np.where(
    merged_df['Volume_df'] == 0,
    merged_df['Volume_dfagg'] / grouped.transform('size'), # If the original Volume is 0, then the prorated Volume is the aggregated Volume divided by the number of rows in the group
    # else,if we had volume for a given service task type (STT), then calculate the percentage of this STT for the ST they belong to and refer to the Volume for the group; in other words, the diistributed Volume is the aggregated Volume of the group multiplied by the percentage of the original Volume for each member in the subgroup
    df.apply(lambda row: dfagg.loc[(dfagg['Fiscal Year'] == row['Fiscal Year']) & (dfagg['Month'] == row['Month']) & (dfagg['Region'] == row['Region']) & (dfagg['Area'] == row['Area'])  & (dfagg['ServiceLine'] == row['ServiceLine']) & (dfagg['ServiceType'] == row['ServiceType']), 'Volume'].values[0] * row['Percentage'] / 100, axis=1)
    
)

final_df = merged_df

print(final_df)

# save to csv
final_df.to_csv('DetailedForecastUpdated.csv', index=False)
